In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/15 14:04:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

--2022-12-15 14:04:05--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.244.115.202, 18.244.115.107, 18.244.115.220, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.244.115.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308924937 (295M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-01.parquet.1’

fhvhv_tripdata_2021 100%[===================>] 294.61M  48.4MB/s    in 6.6s    

utime(fhvhv_tripdata_2021-01.parquet.1): No such file or directory
2022-12-15 14:04:12 (44.8 MB/s) - ‘fhvhv_tripdata_2021-01.parquet.1’ saved [308924937/308924937]



In [4]:
!wc -l fhvhv_tripdata_2021-01.parquet #Parquet format reduces lines used

1006794 fhvhv_tripdata_2021-01.parquet


In [5]:
df = spark.read \
    .parquet('fhvhv_tripdata_2021-01.parquet')

In [6]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DO

As noticed data types are configured correctly due to parquet incode

In [7]:
df.count()

11908468

In [13]:
schema = df.schema
schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [9]:
### if needing to change schema
# !head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv # read partition of huge data avoid memory usage
# import pandas as pd
# df_pandas = pd.read_csv('head.csv')
# df_pandas.dtypes
# spark.createDataFrame(df_pandas).schema

from pyspark.sql import types
# ### insert here the names with desired types:
# schema = types.StructType([
#     types.StructField('hvfhs_license_num', types.StringType(), True),
#     types.StructField('dispatching_base_num', types.StringType(), True),
#     types.StructField('pickup_datetime', types.TimestampType(), True),
#     types.StructField('dropoff_datetime', types.TimestampType(), True),
#     types.StructField('PULocationID', types.IntegerType(), True),
#     types.StructField('DOLocationID', types.IntegerType(), True),
#     types.StructField('SR_Flag', types.StringType(), True)
# ])
# df = spark.read \
#     .schema(schema) \
#     .parquet('fhvhv_tripdata_2021-01.parquet')

# df = df.repartition(24) ###clustering for spark to write files more efficient separated
# df.write.parquet('fhvhv/2021/01/')
# df = spark.read.parquet('fhvhv/2021/01/') ###Combine all parquet in this folder


In [15]:
s_df = spark.read \
    .schema(schema) \
    .parquet('fhvhv_tripdata_2021-01.parquet')
s_df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DO

In [10]:
from pyspark.sql import functions as F ### contains functions to be used plus making udf

In [11]:
### ex of any function that can be complex as SQL
def ex_python_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'


In [12]:
### adding the function to be udf
ex_python_stuff_udf = F.udf(ex_python_stuff, returnType=types.StringType())
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', ex_python_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()
###ability to apply the function on spark df

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  a/a7a| 2021-01-01|  2021-01-01|         230|         166|
|  a/a7a| 2021-01-01|  2021-01-01|         152|         167|
|  e/acc| 2021-01-01|  2021-01-01|         233|         142|
|  e/acc| 2021-01-01|  2021-01-01|         142|         143|
|  e/acc| 2021-01-01|  2021-01-01|         143|          78|
|  e/9ce| 2021-01-01|  2021-01-01|          88|          42|
|  e/9ce| 2021-01-01|  2021-01-01|          42|         151|
|  e/acc| 2021-01-01|  2021-01-01|          71|         226|
|  e/b3b| 2021-01-01|  2021-01-01|         112|         255|
|  e/b3b| 2021-01-01|  2021-01-01|         255|         232|
|  e/b3b| 2021-01-01|  2021-01-01|         232|         198|
|  s/b13| 2021-01-01|  2021-01-01|         113|          48|
|  s/b13| 2021-01-01|  2021-01-01|         239|          75|
|  s/af0| 2021-01-01|  2